In [1]:
def get_query_dictionary(query_arg):
    """Creates a query dictionary with all of the parameters included in the query string.\n
    e.g. {token : 'demotoken', stid : 'NHMU', etc}.

    :param query_arg: query string to parse into dictionary
    :type query_arg: str
    :return: dictionary with query parameters
    :rtype: dict
    """
    params = query_arg.split(sep='&')

    keys = [arg.split('=')[0] for arg in params[1:] if arg]
    values = [arg.split('=')[1] for arg in params[1:] if arg]

    _dict = dict(zip(keys, values))

    if 'accum_hours' in _dict:
        _dict['accum_hours'] = _dict['accum_hours'].split(',')

    return _dict

In [2]:
def make_dataframe(query_string):
    """Constructs a dataframe from the query passed, assumes that the query is for pmode totals or last.

    :param query_string: string of the api query to make the dataframe out of
    :type query_string: str
    :return: dataframe with accumulated precip columns
    :rtype: pandas.DataFrame
    """
    resp = requests.get(query_string)

    # Any errors in the query
    assert resp.status_code == 200, "Server Error or URL Error"
    values_dict = json.loads(resp.text)
    if int(values_dict['SUMMARY']['RESPONSE_CODE']) == 2:
        print(values_dict['SUMMARY']['RESPONSE_MESSAGE'])
        return pd.DataFrame()

    assert int(values_dict['SUMMARY']['RESPONSE_CODE']) == 1, values_dict['SUMMARY']['RESPONSE_MESSAGE'] + '\n' + \
                                                              query_string
    # Success!
    df = json_normalize(values_dict['STATION'])
    # Get the arguments of the API call e.g. end, start, pmode, etc.
    query_dictionary = get_query_dictionary(query_string)

    # pmode last and intervals is not supported.
    if query_dictionary['pmode'] == 'last':
        print('pmode: last is not supported please use pmode totals using a start and end time.')
        exit(1)

    elif query_dictionary['pmode'] == 'intervals':
        print('pmode: intervals is not supported please use pmode totals using a start and end time.')
        exit(1)

    # Totals
    elif query_dictionary['pmode'] == 'totals':
        # parse the API call's start and end and compute a timedelta
        start = datetime.strptime(query_dictionary['start'], '%Y%m%d%H%M')
        end = datetime.strptime(query_dictionary['end'], '%Y%m%d%H%M')
        delta = end.timestamp() - start.timestamp()
        
        # initialize new columns to be added to the dataframe
        new_col = np.full(df.shape[0], np.nan, dtype='float64')
        new_date_col = np.full(df.shape[0], 0, dtype='int')
        new_count_col = np.full(df.shape[0], 0, dtype='int')
        
        # Unwrap the nested data
        for i, row in df.iterrows():
            if len(row['OBSERVATIONS.precipitation']) > 0:
                _dict = row['OBSERVATIONS.precipitation'][0]
                # time difference of reporting interval
                new_date_col[i] = (int(_dict['last_report']) - int(_dict['first_report']))
                # precip total
                new_col[i] = _dict['total']
                # count of obs
                new_count_col[i] = _dict['count']
        
        # Make the new columns from the unwrapped data
        df['ACCUM_' + str(int(delta/86400)).strip() + '_DAYS'] = new_col
        df['EPOCH_TIMEDELTA'] = new_date_col
        df['COUNT'] = new_count_col
        
        # filter out an observations that are not long enough
        df = df[abs(df['EPOCH_TIMEDELTA'] - delta) < .1*delta]
        
    df = df.drop(columns=['OBSERVATIONS.precipitation', 'PERIOD_OF_RECORD.start', 'PERIOD_OF_RECORD.end'])
    df = df.apply(pd.to_numeric, errors='ignore')
    return df